LangChain Tool
```
Tool : llm에이전트가 수행할 수 있는 외부기능이나 api를 말함(날씨정보나 웹검색, 계산기)
이유 : gpt는 최신정보에 접근할 수 없다. 
구조 : 이름 설명 함수실행로직
툴 호출 방식 : GPT-4 모델은 OpenAI의 함수호출 기능을 통해 툴을 직접 호출할 수 있다.
```
외부 API 호출 Tool 구현(날씨API, 뉴스API)

In [24]:
%pip install langchain-core langchain-openai openai python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [25]:
# .env 로드
from dotenv import load_dotenv
load_dotenv()
import os

print(os.getenv("OPEN_WEATHER_MAP")[:5])
print(os.getenv("NEWSAPI_API_KEY")[:5])


66867
647e2


In [26]:
# 날씨정보 조회 함수
from pydantic import BaseModel, Field

# 입력 스키마
class WeatherInput(BaseModel):
    city: str = Field(description="날씨를 조회할 도시 이름(영문)")

def get_weather(city: str) -> str:
    import requests
    api_key = os.getenv("OPEN_WEATHER_MAP")
    url = f"http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # 섭씨 온도로 받기
        'lang': 'kr',  # 한국어로 응답 받기
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if data.get("cod") != 200:
        return f"날씨 정보를 가져오는 데 실패했습니다"
    temp = data['main']['temp']
    desc = data['weather'][0]['description']
    return f"{city}의 현재 기온은 {temp}도이며, 날씨는 {desc}입니다."

In [27]:
# pydantic 모델을 사용하여 입력 검증
class WeatherInput(BaseModel):
    city: str = Field(description="날씨를 조회할 도시 이름(영문)")

class MyClass:
    def __init__(self, ab: str):
        self.city = ab

MyClass(ab=100) # 사용자가 의도한 문자열이 전달되지 않아도 python은 타입에러를 발생시키지 않음 # 블로그
# result = WeatherInput(city=100)
# result.city

In [28]:
get_weather("Seoul")

'Seoul의 현재 기온은 24.76도이며, 날씨는 연무입니다.'

뉴스검색

In [29]:
class NewsInput(BaseModel):
    keyword: str = Field(description="뉴스를 조회할 키워드")

def get_news(keyword: str) -> str:
    import requests
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        'q': keyword,
        'apiKey': api_key,
        'language': 'ko',  # 한국어 뉴스
        'sortBy': 'publishedAt',  # 최신 뉴스 우선
        'pageSize': 1,  # 최대 n개 뉴스
    }
    response = requests.get(url, params=params)
    data = response.json()
    articles = data.get('articles')
    if not articles:
        return "뉴스를 찾을 수 없습니다."
    
    # 가장 첫번째 뉴스 선택
    top_news = articles[0]
    title = top_news.get('title', '제목 없음')
    source = top_news.get('source', {}).get('name', '출처 없음')
    return f'{keyword}에 대한 최신 뉴스:"{title}"(출처: {source})'

In [30]:
print(get_news('OpenAI'))

OpenAI에 대한 최신 뉴스:"AI가 대신 쇼핑하고 예약까지…에이전트 경제 시대 온다"(출처: Zdnet.co.kr)


LangChain Tool 객체로 변환
```
get_weather, get_news 함수를 Tool로 등록
```

In [31]:
from langchain_core.runnables import RunnableLambda
# 날씨함수 runnable -> tool로 변환
weather_runnable = RunnableLambda(lambda x: get_weather(x['city']))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시의 현재 날씨 정보를 반환합니다.",
    args_schema=WeatherInput,
)
news_runnable = RunnableLambda(lambda x: get_news(x['keyword']))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드에 대한 최신 뉴스를 반환합니다.",
    args_schema=NewsInput,
)
print(f'tool 이름: {weather_tool.name} 설명 : {weather_tool.description}')
print(f'tool 이름: {news_tool.name} 설명 : {news_tool.description}')

tool 이름: get_weather 설명 : 도시의 현재 날씨 정보를 반환합니다.
tool 이름: get_news 설명 : 키워드에 대한 최신 뉴스를 반환합니다.


In [32]:
# 툴없이
from langchain_openai import ChatOpenAI
# 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
# 출력 파서
from langchain_core.output_parsers import StrOutputParser
user_question = "서울의 날씨 알려줘"
# 사용자 질문을 처리하는 프롬프트 템플릿
user_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 날씨와 뉴스 정보를 제공하는 AI입니다."),
    ("human", "{question}"),
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = user_prompt | llm | StrOutputParser()
# 사용자 질문을 처리하는 체인 실행
result = chain.invoke({"question": user_question})
print(result)

현재 서울의 날씨 정보를 실시간으로 제공할 수는 없지만, 일반적으로 서울의 날씨는 계절에 따라 다릅니다. 봄(3월~5월)에는 온화하고 꽃이 피며, 여름(6월~8월)에는 덥고 습합니다. 가을(9월~11월)에는 선선하고 단풍이 아름답고, 겨울(12월~2월)에는 춥고 눈이 내릴 수 있습니다. 

정확한 날씨 정보를 원하신다면, 기상청 웹사이트나 날씨 앱을 확인해 보시기 바랍니다.


LLM 최종 답변 생성 요청

In [34]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# 프롬프트구성 : 시스템메시지 + 사용자 질문 + 결과를 맥락으로 제공
weather_info = get_weather("Seoul")
system_prompt = '당신은 유용한 AI 어시스턴스입니다. 사용자 질문에 맞게 제공된 정보를 활용해 답변하세요'
tool_info_prompt = f"도구가 제공한 추가 정보:{weather_info}"
ask_prompt = '위 정보를 참고해서 사용자 질문에 답변하세요'
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question),
    AIMessage(content=tool_info_prompt), # 도구 결과를 마치 ai의 답변처럼 추가
    HumanMessage(content=ask_prompt)
]
result = llm(messages)
print(result.content)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_19924\244140487.py:13: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(messages)


현재 서울의 날씨는 연무이며, 기온은 약 24.76도입니다. 날씨가 흐리거나 뿌연 상태일 수 있으니 외출 시 참고하시기 바랍니다.


여러 tool을 연결하도록 에이전트 구성

In [ ]:
# 에이전트를 생성하고 등록
from langchain.agents import initialize_agent, AgentType
# 앞에서 만든 tool 객체 리스트를 준비
tools = [weather_tool, news_tool]